# Marelle Board import
* Getting players info dict => `MarelleBoard.players`
* 3 phases : place, move and end => `MarelleBoard.phase`
* Reset the board => `MarelleBoard.initialize_game()`
* Printing the board => `MarelleBoard.print_board()`
* Id:Action dict => `MarelleBoard.action_list_by_id`
* Action:Id dict => `MarelleBoard.action_id_by_action`
* Get the board state => `MarelleBoard.get_state()`
* Play an action => `MarelleBoard.play_action(action_id, player)`
* Get legal action ids => `MarelleBoard.get_legal_action_ids(player)`
* Check if game ended (returns 0 if not ended or winning player id) : `MarelleBoard.check_if_end()`

In [1]:
from maRL_board import MarelleBoard

In [2]:
board = MarelleBoard()
board.initialize_game()
board.phase

'place'

In [6]:
board.print_board()


        Phase : place
        Placed tokens : P1 : 0 / P2 : 0 
        Tokens on board : P1 : 0 / P2 : 0 
        0--------------0--------------0
        |              |              |
        |    0---------0---------0    |
        |    |         |         |    |
        |    |    0----0----0    |    |
        |    |    |         |    |    |
        0----0----0         0----0----0                 
        |    |    |         |    |    |
        |    |    0----0----0    |    |
        |    |         |         |    |
        |    0---------0---------0    |
        |              |              |
        0--------------0--------------0
        


96

In [13]:
board=MarelleBoard()
board.print_board()
a=board.get_legal_action_ids(1)[0]
board.play_action(a,1)
board.print_board()
b=board.get_legal_action_ids(-1)[2]
print(board.action_list_by_id[b])
board.play_action(b,-1)
board.print_board()


        Phase : place
        Placed tokens : P1 : 0 / P2 : 0 
        Tokens on board : P1 : 0 / P2 : 0 
        0--------------0--------------0
        |              |              |
        |    0---------0---------0    |
        |    |         |         |    |
        |    |    0----0----0    |    |
        |    |    |         |    |    |
        0----0----0         0----0----0                 
        |    |    |         |    |    |
        |    |    0----0----0    |    |
        |    |         |         |    |
        |    0---------0---------0    |
        |              |              |
        0--------------0--------------0
        

        Phase : place
        Placed tokens : P1 : 1 / P2 : 0 
        Tokens on board : P1 : 1 / P2 : 0 
        0--------------0--------------0
        |              |              |
        |    0---------0---------0    |
        |    |         |         |    |
        |    |    0----0----0    |    |
        |    |    |         |    |    |


In [ ]:
import gym
from gym import spaces


class MarelleGymEnv(gym.Env):
    """Custom Environment that follows gym interface"""
    metadata = {'render.modes': ['human']}
    def __init__(self):
        super(CustomEnv, self).__init__()    
    
    # Define action and observation space
    # They must be gym.spaces objects
        self.action_space = spaces.Discrete(N_ACTIONS)    # Example for using image as input:
        self.observation_space = spaces.Box(low=-1, high=1, shape=(3, 8), dtype=np.uint8)

  
  def step(self, action):
    return
    # Execute one time step within the environment
  
  def reset(self):
    return
    # Reset
  
  def render(self, mode='human', close=False):
    return
    # print the board
